In [1]:
from keras.layers import Input, Dense, Lambda, Flatten, Activation, Merge, Concatenate, Add
from keras import layers
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
from scipy.stats import sem
from scipy.spatial.distance import squareform, pdist
from keras import backend as K
from keras.models import load_model
import tensorflow as tf

from models import vgg
from models.cvae import CVAE
from models.cvae_mod import CVAE as CVAE_mod
from utils.losses import von_mises_log_likelihood_np
from utils.angles import deg2bit, bit2deg
from utils.losses import mad_loss_tf, cosine_loss_tf, von_mises_loss_tf, maad_from_deg
from utils.losses import gaussian_kl_divergence_tf, gaussian_kl_divergence_np
from utils.losses import gaussian_log_likelihood_np, gaussian_log_likelihood_scipy, gaussian_log_likelihood_tf
from utils.losses  import von_mises_log_likelihood_tf, von_mises_log_likelihood_np
from utils.sampling import sample_multiple_gauassians_np
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

Using TensorFlow backend.


In [4]:
xtr, ytr_deg, xval, yval_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True, verbose=1)

image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yval_bit = deg2bit(yval_deg)
yte_bit = deg2bit(yte_deg)
yte_rad = np.deg2rad(yte_deg)

image_height, image_width, n_channels = xtr.shape[1:]
flatten_x_shape = xtr[0].flatten().shape[0]
phi_shape = yte_bit.shape[1]

Number of train samples: 6916
Number of validation samples: 874
Number of test samples: 904


Gaussian log-likelihood

$\log L(x)= -\frac{1}{2} \left( \log (|\boldsymbol\Sigma|\,) + (\mathbf{x}-\boldsymbol\mu)^{\rm T}\boldsymbol\Sigma^{-1}(\mathbf{x}-\boldsymbol\mu) + n\log(2\pi) \right)$

In case $\boldsymbol\Sigma = diag(\sigma_1^2, \dots, \sigma_n^2)$

$\log L(x)= -\frac{1}{2} \left( (\sum_{i=1}^{n}{\log \sigma^2_i}) + \sum_{i=1}^{n}{\frac{(x_i-\mu_i)^2}{\sigma^2_i}} + n\log(2\pi) \right)$


In [5]:
def get_model_samples(cvae, x, y, n_samples=10):
    
    n_points = x.shape[0]
    
    x_tiled = np.repeat(x_ex, repeats=n_samples, axis=0)
    y_tiled = np.repeat(y_ex, repeats=n_samples, axis=0)
    
    output = cvae.get_full_output(x, y)
    mu_prior = output['mu_prior']
    std_prior = np.exp(output['log_sigma_prior'] / 2)
    mu_encoder = output['mu_encoder']
    std_encoder = np.exp(output['log_sigma_encoder'] / 2)
    u_encoder = output['u_encoder']
    mu_pred = output['mu_pred']
    
    u_samples = np.zeros([n_points, n_samples, cvae.n_u])
    
    mu_preds = np.zeros([n_points, n_samples, 2])
    kappa_preds = np.zeros([n_points, n_samples, 1])
        
    for sid in range(0, n_samples):
        
        kappa_preds[:, sid, :] = output['kappa_pred']
        
        #vm_likelihood[:, sid] = np.squeeze(np.exp(von_mises_log_likelihood_np(y, mu_pred, kappa_pred)))
        u_samples[:, sid, :] = u_encoder
    
    prior_log_likelihood =  gaussian_log_likelihood_np(mu_prior, std_prior, u_samples)
    encoder_log_likelihood = gaussian_log_likelihood_np(mu_encoder, std_encoder, u_samples)
    
    sample_weight = np.exp(prior_log_likelihood - encoder_log_likelihood)
    
    importance_likelihoods = np.log(np.mean(vm_likelihood*sample_weight, axis=1))
    
    return u_samples, mu_pred

#### Improtance sampling
$ p(\phi| x) \approx \frac{1}{S} \sum_{i=1}^{S}{\frac{ p(\phi | x, u_i) p(u_i|x)  }{ q(u_i| x, \phi)}}$

where

$ p(u|x) \sim \mathcal{N}(\mu_1(x), \sigma_1(x)) $

$ q(u|x,\phi) \sim \mathcal{N}(\mu_2(x, \phi), \sigma_2(x, \phi)) $

$ p(\phi|x, u) \sim \mathcal{VM}(\mu(x,u), \kappa(x,u)) $


In [42]:
n_u = 8
cvae_ckpt_path = 'logs/best_models/cvae/2/cvae.full_model.trial_0.best.weights.hdf5'

cvae = CVAE_mod(n_hidden_units=n_u, n_samples=128)
cvae.full_model.load_weights(cvae_ckpt_path)

In [43]:
out = cvae.full_model.predict([xte, yte_bit])
out = cvae.parse_output_np(out)

In [38]:
out.shape

AttributeError: 'dict' object has no attribute 'shape'

In [44]:
out['log_sigma_prior'][0]

array([ 0.02635976,  0.05754765,  0.05291699,  0.06663045,  0.04948011,
        0.03372288,  0.0700205 ,  0.03959129], dtype=float32)

In [7]:
out['log_sigma_encoder'][0]

array([ 0.03050286,  0.04987274,  0.04818736,  0.07788163,  0.0470078 ,
        0.04491068,  0.07260608,  0.05432089], dtype=float32)

In [9]:
out['mu_prior'][0]

array([-0.01503756,  0.00639821, -0.00642541, -0.02498445, -0.01205181,
        0.0185044 , -0.0402074 ,  0.00627557], dtype=float32)

In [8]:
out['mu_encoder'][0]

array([ 0.0034244 ,  0.02234553, -0.00068222, -0.03396796, -0.01267993,
        0.01617561, -0.01503824,  0.00521719], dtype=float32)

In [12]:
bit2deg(ytr_bit)

array([ 210.,   32.,  165., ...,  193.,  186.,  179.])

In [41]:
np.mean(maad_from_deg(yte_deg, bit2deg(out['mu_preds'][:,0,:])))

26.453822414431951

In [36]:
out.shape

(904, 384)

In [45]:
np.mean(cvae.importance_log_likelihood_np(yte_bit, cvae.full_model.predict([xte, yte_bit])))

-0.70381010128750288

In [46]:
np.mean(cvae.importance_log_likelihood_np(yval_bit, cvae.full_model.predict([xval, yval_bit])))

-0.62684722476709509

In [ ]:
np.mean(cvae.importance_log_likelihood_np(yval_bit, cvae.full_model.predict([xval, yval_bit])))

In [25]:
out_ph = tf.placeholder(dtype=tf.float32, shape=[None, 152])
y_ph = tf.placeholder(dtype=tf.float32, shape=[None, 2])

impll_tf = cvae.importance_log_likelihood_tf(y_ph, out_ph)

In [26]:
with tf.Session() as sess:
    ll = impll_tf.eval(feed_dict={out_ph:out, y_ph:y})

In [27]:
ll

1.8896326